# Initialize stuff first

In [1]:
print("listing package versions... ")
import sys
print("python:", sys.version)
import pickle as pkl
import seaborn as sns
print("seaborn:", sns.__version__)
import matplotlib as mpl
import matplotlib.pyplot as plt
print("matplotlib:", mpl.__version__)
import numpy as np
print("numpy:", np.__version__)
import numba as nb
print("numba:", nb.__version__)
import scipy as sp
import scipy.integrate as sp1
import scipy.interpolate as sp2
import scipy.misc as sp3
import scipy.optimize as sp4
print("scipy:", sp.__version__)
import os
from pathlib import Path
%matplotlib inline

listing package versions... 
python: 3.7.0 (default, Jun 28 2018, 13:15:42) 
[GCC 7.2.0]
seaborn: 0.9.0
matplotlib: 2.2.3
numpy: 1.15.1
numba: 0.39.0
scipy: 1.1.0


In [2]:
mpl.rcParams['figure.dpi'] = 100
mpl.rcParams['figure.figsize'] = 8,8

sns.set_context("paper")
sns.set(font='serif')
sns.set_style("white", {
        "font.family": "serif",
        "font.serif": ["Times", "Palatino", "serif"]})

pardir=Path(os.getcwd()).parent.parent
datadir=str(pardir) +'/Data'

mycmap=sns.cubehelix_palette(n_colors=50, start=1, rot=4, light=0.7, dark=0)

In [3]:
# my homebrew 2d interpolation function
@nb.autojit(nopython=True)
def int2d(x,y,matrix):
    a,b=matrix.shape
    left=np.int(min(max(np.int(x),0),a-2))
    right=left+1
    bottom=np.int(min(max(np.int(y),0),b-2))
    top=bottom+1
    x=right-x
    y=top-y
    return x*y*matrix[left,bottom]+(1-x)*y*matrix[right,bottom]+x*(1-y)*matrix[left,top]+(1-x)*(1-y)*matrix[right,top]

# this copy to pass into the integrator (scipy won't take the raw function)
@nb.autojit()
def int2dslow(x,y,matrix):
    a,b=matrix.shape
    left=np.int(min(max(np.int(x),0),a-2))
    right=left+1
    bottom=np.int(min(max(np.int(y),0),b-2))
    top=bottom+1
    x=right-x
    y=top-y
    return x*y*matrix[left,bottom]+(1-x)*y*matrix[right,bottom]+x*(1-y)*matrix[left,top]+(1-x)*(1-y)*matrix[right,top]

In [4]:
@nb.autojit()
def create_qth(dist_qth,dist_rmcq,right,fc,w,scale): 
    for i in range(1,100*fc+1):
        q=i/fc
        for j in range(0,right):    
            th=j/(fc*scale)
            c=np.exp(-th+w*np.log(q)+np.log(100))
            rmc=100-c
            jac=c
            if rmc>=0 and rmc <=99:
                dist_qth[i,j]=int2d(i,rmc*fc,dist_rmcq)*jac

In [5]:
@nb.autojit(nopython=True)
def create1(Fp1,right,fc): 
    for j in range(0,right):
        for i in range(0,100*fc):
            Fp1[i+1,j]+=Fp1[i,j]

@nb.autojit(nopython=True)
def create2(Fp2,right,fc): 
    for i in range(0,100*fc+1):
        for j in range(0,right-1):
            Fp2[i,j+1]+=Fp2[i,j]

@nb.autojit(nopython=True)
def create0(F,right,fc): 
    for i in range(0,100*fc+1):
        for j in range(0,right-1):
            F[i,j+1]+=F[i,j]
            
@nb.autojit(nopython=True)
def createZ(Z,F,right,freq,powr,fc): 
    FF=F/np.max(F)
    for i in range(0,100*fc+1):
        for j in range(0,right):
            num=0
            den=0
            for k in range(0,len(powr)):
                num+=np.power(FF[i,j],powr[k]-2)*freq[k]
                den+=np.power(FF[i,j],powr[k]-2)*freq[k]*(powr[k]-1)
            Z[i,j]=num/den

@nb.autojit(nopython=True)
def create3(Z,F,Fp1,Fp2,rhs,rhsfake,right,fc,scale,w): 
    for i in range(1,100*fc+1):
        q=i/fc
        aq=-100/np.power(q,w)
        myright=min(int(round(fc*scale*(w*np.log(q)+np.log(100)))),right)
        myleft=max(0,int(round(fc*scale*(w*np.log(q)))))
        for j in range(myleft,myright):
            th=-np.exp(np.log(100)-j/(fc*scale)) 
            if th>aq:
                den=-Fp2[i,j]+100*w*Z[i,j]*F[i,j]/((th-aq)*fc*np.power(q,w+1))
                if den!=0:
                    my=Fp1[i,j]/den
                    if my>=0 and my<5:
                        rhs[i,j]=my
                        rhsfake[i,j]=1
            rhs[0,0]=1/2
            rhs[0,1]=1/2
            
@nb.autojit(nopython=True)
def create8(reflect7,reflect8,myinv,myfor,F,xmax,ymax,fc,right,scale,w): 
    for i in range(1,100*fc+1):
        q=i/fc
        aq=-100/np.power(q,w)
        myright=min(int(round(fc*scale*(w*np.log(q)+np.log(100)))),right)
        myleft=max(0,int(round(fc*scale*(w*np.log(q)))))
        for j in range(myleft,myright):
            if i<ymax:
                if j<xmax:
                    inv=myinv[i]
                    if inv>=j:
                        reflect8[i,j]=F[i,inv]
                    else:
                        reflect8[i,j]=F[myfor[j],j]
                else:
                    reflect8[i,j]=F[100*fc,j]
            else:
                reflect8[i,j]=F[i,right-1]
                
@nb.autojit(nopython=True)
def create9(reflect8,reflect9, l, powers, frequencies, right, fc): 
    for k in range(0,l):
        pk=powers[k]-1
        fk=frequencies[k]
        for i in range(0,100*fc+1):
            myright=right#i+int(round(100*fc*ww))
            for j in range(0,myright):
                reflect9[i,j]+=(reflect8[i,j]**pk)*fk
                
@nb.autojit(nopython=True)
def myhelp(rrr, qqq, fc, scale, w, right): 
    for i in range(1,100*fc+1):
        q=i/fc
        myright=min(int(round(fc*scale*(w*np.log(q)+np.log(100)))),right)
        myleft=max(0,int(round(fc*scale*(w*np.log(q)))))
        for j in range(myleft,myright):
            rrr[i,j]=np.power(q,w)
            qqq[i,j]=q 

@nb.autojit(nopython=True)
def createR(reflect9,reflect9help,reflect9help2,rebate,myinv,right,fc,scale,w): 
    end=min(100*fc+1,len(myinv))
    for i in range(1,end):
        q=i/fc
        aq=-100/np.power(q,w)
        myright=right
        for j in range(0,myright-1):
            jloc=j/(fc*scale)
            for k in range(j,myright-1):
                reflect9help2[i,k+1]+=np.exp(np.log(100)-jloc)*reflect9help[i,j]/(fc*scale)
        for j in range(0,myright):
            th=-np.exp(np.log(100)-j/(fc*scale)) 
            rebate[i,j]=max((th-aq)*reflect9[i,j]-reflect9help2[i,j],0)
            
@nb.autojit(nopython=True)
def mymultiply(reflect9,rebate,quality): 
    mysum=np.sum(reflect9)
    EQ=np.sum(np.multiply(reflect9,quality))/mysum
    ER=np.sum(np.multiply(reflect9,rebate))/mysum
    return ER,EQ

In [6]:
fc=8
design='loglinear'
for industry in ["leg", "mil", "sci", "edu", "tec"]:
    print('industry=',industry)
    for weight in ['55','80']:
        print('weight=',weight)
        mydir=datadir+"/derive/"+industry+weight+"/"
        [PART,DATA]=pkl.load(open(datadir+'/derive/'+industry+weight+"j.pkl", "rb"))
        dist_rmcq=pkl.load(open(mydir+industry+weight+"j_rcmq_"+str(fc)+".pkl", "rb"))

        mpl.rcParams['figure.figsize'] = 8,8

        powr=np.array(PART[0])
        freq=np.array(PART[1])
        freq=freq/np.sum(freq)

        curve={}
        for w in list(np.array(range(16,280))/20):
            w=w/8  
            #w=1300
            #w=w/1000

            scale=30
            right=int(round(fc*scale*(w+1)*np.log(100)))
            dist_qth=np.zeros((100*fc+1,right),dtype=float)

            boundary1=[fc*np.exp((j/(fc*scale)-np.log(100))/w) for j in range(0,right)]
            boundary2=[fc*np.exp((j/(fc*scale))/w) for j in range(0,right)]

            create_qth(dist_qth,dist_rmcq,right,fc,w,scale)
            dist_qth=fc*fc*scale*dist_qth/np.sum(dist_qth)

            #do not rescale these, it's important
            Fp1=dist_qth.copy(); create1(Fp1,right,fc)
            Fp2=dist_qth.copy(); create2(Fp2,right,fc)
            F=Fp1.copy(); create0(F,right,fc)
            #print('this should be equal to one:', np.max(F/(fc*fc*scale)))

            Z=np.zeros((100*fc+1,right),dtype=float) 
            createZ(Z,F,right,freq,powr,fc)        

            rhsfake=np.zeros((100*fc+1,right),dtype=float)                            
            rhs=np.zeros((100*fc+1,right),dtype=float)
            create3(Z,F,Fp1,Fp2,rhs,rhsfake,right,fc,scale,w)   

            #print(np.min(rhs),np.max(rhs))

            mydic={}

            def myfun(x,y):
                return int2dslow(y,x,rhs)

            myode=sp1.ode(myfun, jac=None)

            y0=1*fc
            x0=1

            myode.set_initial_value(y0,x0)
            myode.set_integrator('dopri5', nsteps=5000)

            dt=1
            x=[x0-2,x0-1]
            y=[y0-2,y0-1]

            while y[-1] < 100*fc+1 and x[-1] < right:
                newy=myode.integrate(myode.t+dt)[0]
                if newy-y[-1]>10*dt or (newy==y[-1] and y[-1]>2*fc):
                    x=x+[x[-1]]
                    y=y+[y[-1]+dt]
                    myode.set_initial_value(y[-1], x[-1])
                else:
                    x=x+[myode.t+dt]
                    y=y+[newy]

            mydic[int(round(1000*w))]=[np.array(x),np.array(y)]

            fstrat = sp2.interp1d([0]+list(x),[0]+list(y))
            fstrat_inv = sp2.interp1d([0]+list(y),[0]+list(x))

            ymax=int(max(y))
            #print('ymax=',ymax)

            xmax=int(max(x))
            #print('xmax=',xmax)

            myinv=[]
            for i in range(0,ymax):
                myinv+=[int(fstrat_inv(i))]
            myinv=np.array(myinv)

            myfor=[]
            for i in range(0,xmax):
                myfor+=[int(fstrat(i))]
            myfor=np.array(myfor)



            dist_pseudo=np.zeros((100*fc+1,right),dtype=float)

            create8(dist_qth,dist_pseudo,myinv,myfor,F,xmax,ymax,fc,right,scale,w)
            dist_pseudo=dist_pseudo/np.max(dist_pseudo)

            frequencies=np.array(PART[1])/np.sum(PART[1])
            powers=np.array(PART[0])
            l=len(powers)    

            num=0
            for i in range(0,len(powers)):
                num+=powers[i]*frequencies[i]

            #print('num=',num)

            dist_power=np.zeros((100*fc+1,right),dtype=float)

            create9(dist_pseudo,dist_power, l ,powers, frequencies, right, fc)

            #print(np.max(dist_power))

            rrr=np.zeros((100*fc+1,right),dtype=float)      
            qqq=np.zeros((100*fc+1,right),dtype=float)

            myhelp(rrr, qqq, fc, scale, w, right)

            quality=np.multiply(dist_power,qqq)

            rebate=np.zeros((100*fc+1,right),dtype=float)

            dist_power_help=dist_power.copy()
            dist_power_help2=np.zeros((100*fc+1,right),dtype=float)

            createR(dist_power,dist_power_help,dist_power_help2,rebate,myinv,right,fc,scale,w)

            rebate=np.multiply(rebate,rrr)

            ER,EQ=mymultiply(dist_qth,rebate,quality)
            sys.stdout.write('w='+str(w)+': ER='+str(round(num*ER,2))+', EQ='+str(round(num*EQ,2))+' || ')

            curve[int(round(1000*w))]=[num*ER,num*EQ]
        print('saving...')        
        addr=mydir+industry+weight+"j_curve_"+design+".pkl"
        pkl.dump(curve,open(addr, "wb"))

industry= leg
weight= 55
w=0.1: ER=36.26, EQ=70.33 || w=0.10625: ER=36.26, EQ=70.51 || w=0.1125: ER=36.25, EQ=70.69 || w=0.11875: ER=36.23, EQ=70.87 || w=0.125: ER=36.22, EQ=71.04 || w=0.13125: ER=36.2, EQ=71.22 || w=0.1375: ER=36.17, EQ=71.39 || w=0.14375: ER=36.15, EQ=71.56 || w=0.15: ER=36.12, EQ=71.73 || w=0.15625: ER=36.09, EQ=71.9 || w=0.1625: ER=36.05, EQ=72.07 || w=0.16875: ER=36.01, EQ=72.24 || w=0.175: ER=35.98, EQ=72.4 || w=0.18125: ER=35.93, EQ=72.57 || w=0.1875: ER=35.89, EQ=72.73 || w=0.19375: ER=35.84, EQ=72.89 || w=0.2: ER=35.79, EQ=73.06 || w=0.20625: ER=35.74, EQ=73.21 || w=0.2125: ER=35.68, EQ=73.38 || w=0.21875: ER=35.63, EQ=73.53 || w=0.225: ER=35.56, EQ=73.69 || w=0.23125: ER=35.5, EQ=73.84 || w=0.2375: ER=35.43, EQ=74.0 || w=0.24375: ER=35.37, EQ=74.15 || w=0.25: ER=35.29, EQ=74.31 || w=0.25625: ER=35.22, EQ=74.46 || w=0.2625: ER=35.15, EQ=74.61 || w=0.26875: ER=35.07, EQ=74.76 || w=0.275: ER=34.98, EQ=74.92 || w=0.28125: ER=34.9, EQ=75.07 || w=0.2875: ER=34.82, 